In [1]:
pip install scx>=1.0.7

Note: you may need to restart the kernel to use updated packages.


<h1>Blinky P5</h1>
<p>A continuation of P4</p>
<p>Robert has found another distribution center (DC2) that could be a good fit for Blink’s distribution network, and he would like to incorporate this option to the optimization model. Robert believes that DC2 could be more convenient than DC1. You are happy to run a new optimization model to figure this out. You also realize  that the fixed costs of running each DC will be relevant to this decision and  decide to incorporate them into your model. </p>
<p>Fixed costs:</p>
<p>- DC 1: &#36; 11500</p>
<p>- DC2: &#36; 15500</p>
<p><b>Table 5: Distance (in miles) from each assembly plant to the DC2 </b></p>
<table width="454" height="100" style="width: 30.2828%;">
<tbody>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black; width: 66.5236%;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.2618%;">DC2</td>
</tr>
<tr>
<td width="15%" style="text-align: center; border: 1px solid black; width: 66.5236%;">Assembly Plant 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.2618%;">15</td>
</tr>
<tr>
<td width="15%" style="text-align: center; border: 1px solid black; width: 66.5236%;">Assembly Plant 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 33.2618%;">36</td>
</tr>
</tbody>
</table>
<p><b>Table 6: Distance in Miles - DC2 to Regions</b></p>
<table width="796" height="24" style="width: 30.3495%;">
<tbody>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black; width: 39.6985%;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">Region 3</td>
</tr>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black; width: 39.6985%;">DC</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">135</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">45</td>
<td width="10%" style="text-align: center; border: 1px solid black; width: 19.8492%;">129</td>
</tr>
</tbody>
</table>
<p>The transportation cost from Assembly plants to either DC is still &#36;0.04 (inbound) and from DC to regions is &#36;0.08 (outbound).&nbsp;</p>
<p><strong>Adjust your model considering the newly available information. Your goal will be to minimize the total (fixed + variable) cost.</strong></p>
<a href="https://youtu.be/RsZ_VrKxtMk">Blinky P5 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="//www.youtube.com/embed/RsZ_VrKxtMk" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'A1',
        'destination_name':'DC1',
        'distance': 190,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'A2',
        'destination_name':'DC1',
        'distance': 150,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R1',
        'distance': 15,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R2',
        'distance': 59,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC1',
        'destination_name':'R3',
        'distance': 79,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'A1',
        'destination_name':'DC2',
        'distance': 15,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'A2',
        'destination_name':'DC2',
        'distance': 36,
        'cost_per_mile':0.04,
    },
    {
        'origin_name':'DC2',
        'destination_name':'R1',
        'distance': 135,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC2',
        'destination_name':'R2',
        'distance': 45,
        'cost_per_mile':0.08,
    },
    {
        'origin_name':'DC2',
        'destination_name':'R3',
        'distance': 129,
        'cost_per_mile':0.08,
    },
]

demand = [
    {
        'name':'R1', 
        'demand':2500
    },
    {
        'name':'R2', 
        'demand':4350
    },
    {
        'name':'R3', 
        'demand':3296
    },
]

assembly = [
    {
        'name':'A1',
        'limit':13000,
    },
    {
        'name':'A2',
        'limit':3500,
    },
]

distribution_center = [
    {
        'name': 'DC1',
        'fixed_cost': 11500,
    },
    {
        'name': 'DC2',
        'fixed_cost': 15500,
    },
]

# Some number big enough...
M = 999999

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)
    # Calculate the variable cost of shipping for each item in tranport
    t['cost']=t['distance']*t['cost_per_mile']

In [5]:
for dc in distribution_center:
    dc['use']=Model.variable(name=f"{dc['name']}__use", cat="Binary")

In [6]:
# Initialize the my_model
my_model = Model(name="Blinky22", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=(
        Model.sum([t['amt']*t['cost'] for t in transport])
        +
        Model.sum([dc['use']*dc['fixed_cost'] for dc in distribution_center])
    )
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand']
    )

## Supply Constraint
for a in assembly:
    my_model.add_constraint(
        name=f"{a['name']}__assembly_supply",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==a['name']]) <= a['limit']
    )

## Conservation of Flow Constraint
for dc in distribution_center:
    my_model.add_constraint(
        name=f"{dc['name']}__conservation_of_flow",
        # Set inbound flows for the DC equal to outbound flows
        fn=Model.sum(
            [t['amt'] for t in transport if t['destination_name']==dc['name']]
        ) == Model.sum(
            [t['amt'] for t in transport if t['origin_name']==dc['name']]
        )
    )

## Linking Constraint
for dc in distribution_center:
    my_model.add_constraint(
        name=f"{dc['name']}__linking_constraint",
        # Only allow the flows into the DC to be greater than 0 if the dc use variable is toggled
        fn=Model.sum(
            [t['amt'] for t in transport if t['destination_name']==dc['name']]
        ) <= M * dc['use']
    )

# Solve the my_model
my_model.solve()

In [7]:
my_model.show_formulation()

Blinky22:
MINIMIZE
7.6000000000000005*A1__DC1__amt + 0.6*A1__DC2__amt + 6.0*A2__DC1__amt + 1.44*A2__DC2__amt + 1.2*DC1__R1__amt + 4.72*DC1__R2__amt + 6.32*DC1__R3__amt + 11500*DC1__use + 10.8*DC2__R1__amt + 3.6*DC2__R2__amt + 10.32*DC2__R3__amt + 15500*DC2__use + 0.0
SUBJECT TO
R1__demand: DC1__R1__amt + DC2__R1__amt >= 2500

R2__demand: DC1__R2__amt + DC2__R2__amt >= 4350

R3__demand: DC1__R3__amt + DC2__R3__amt >= 3296

A1__assembly_supply: A1__DC1__amt + A1__DC2__amt <= 13000

A2__assembly_supply: A2__DC1__amt + A2__DC2__amt <= 3500

DC1__conservation_of_flow: A1__DC1__amt + A2__DC1__amt - DC1__R1__amt
 - DC1__R2__amt - DC1__R3__amt = 0

DC2__conservation_of_flow: A1__DC2__amt + A2__DC2__amt - DC2__R1__amt
 - DC2__R2__amt - DC2__R3__amt = 0

DC1__linking_constraint: A1__DC1__amt + A2__DC1__amt - 999999 DC1__use <= 0

DC2__linking_constraint: A1__DC2__amt + A2__DC2__amt - 999999 DC2__use <= 0

VARIABLES
A1__DC1__amt Continuous
A1__DC2__amt Continuous
A2__DC1__amt Continuous
A2__DC2__

In [8]:
# Show the outputs
my_model.show_outputs()

{'objective': 98262.32,
 'status': 'Optimal',
 'variables': {'A1__DC1__amt': 0.0,
               'A1__DC2__amt': 10146.0,
               'A2__DC1__amt': 0.0,
               'A2__DC2__amt': 0.0,
               'DC1__R1__amt': 0.0,
               'DC1__R2__amt': 0.0,
               'DC1__R3__amt': 0.0,
               'DC1__use': 0.0,
               'DC2__R1__amt': 2500.0,
               'DC2__R2__amt': 4350.0,
               'DC2__R3__amt': 3296.0,
               'DC2__use': 1.0}}
